# Funções geradoras
- Normalmente são funções que sabem 'pausar' em vez de 'parar' com o return
- Consiste em uma forma de criar uma função que representa uma expressão geradora
- O `return` segue sendo o fim da função
- As 'pausas' são feitas com `yield`
- São iterators, pois possuem o método iter e next.
- O return neste caso levanta uma exceção de StopIteration.

In [7]:
def generator(n = 0):
    yield 1
    print("Continuando...")
    yield 2
    print("Mais uma...")
    yield 3
    print("Vou terminar!")
    return "Acabou!"

gen = generator()
print(gen)

print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))

<generator object generator at 0x000002068DEE8C40>
1
Continuando...
2
Mais uma...
3
Vou terminar!


StopIteration: Acabou!

- Por ser um iterador, podemos usá-lo em um for:

In [8]:
def generator2(n = 0):
    yield 1
    print("Continuando...")
    yield 2
    print("Mais uma...")
    yield 3
    print("Vou terminar!")
    return "Acabou!"

gen2 = generator2()
for n in gen2:
    print(n)


1
Continuando...
2
Mais uma...
3
Vou terminar!


In [14]:
# Criando um (n for n in range(10))

def generator3(n = 0, maximum = 10):
    while True:
        yield n # O yield faz uma pausa no nosso While, como um input(0
        n += 1

        if n >= maximum:
            return

gen3 = generator3()
for n in gen3:
    print(n)

0
1
2
3
4
5
6
7
8
9


## O que é? 
- Uma maneira especial de criar iteradores em Python.

- Uma função que pode ser pausada e retomada. Em vez de calcular todos os valores de uma vez e retornar uma lista completa (o que consumiria muita memória), uma função geradora produz os valores um por um, "sob demanda".

- A palavra mágica que faz isso acontecer é yield.

### O problema que ela resolve
- Isso é muito vantajoso em relação à memória: Imagine que queremos criar uma lista com todos os números de 1 a 10 milhões:

In [19]:
# Abordagem tradicional
def cria_lista_gigante():
    numeros = []
    for i in range(1, 10_000_001):
        numeros.append(i)
    return numeros

minha_lista = cria_lista_gigante()
# Isso vai consumir uma quantidade ENORME de memória RAM, 
# pois o PC teria que alocar memória para armazenar todos os 
# 10 milhões de números de uma só vez.

In [20]:
# Abordagem com generator function

def cria_gerador_gigante():
    for i in range(1, 10_000_001):
        yield i
# A função não é executada ainda! Apenas cria um objeto gerador.
# É instantâneo e não usa quase nada de memória.

meu_gerador = cria_gerador_gigante()

## Como funciona a mágina do `yield`?
- A palavra-chave `yield` é o coração dos geradores. Ela funciona de forma parecida com o `return`:
    - retorna um valor
    - pausa a execução imediatamente após entregar o valor, salvando todo o seu estado (variáveis locais, posição no loop, etc)
    - retoma de onde parou na próxima vez que for solicitado ao gerador

- Quando não há mais `yield` para executar, o gerador lança uma exceção `StopIteration`, que sinaliza para os loops `for` e outras construções que a iteração terminou.

## Yield from
- Os generators pausam, então são ótimos para executar códigos que possuem determinada ordem
- Porém, podemos ter outro generator que faz uma sequência e, no nosso generator 'principal', usar esse generator em determinado yield.


In [22]:
def gen1():
    yield 1
    yield 2
    yield 3

def gen2():
    yield from gen1()
    yield 4
    yield 5
    yield 6

g = gen2()
for numero in g:
    print(numero)

1
2
3
4
5
6


- Basicamente, **o `yield from` é uma forma de um gerador delegar parte do seu trabalho para outro gerador (ou qualquer iterável).
### Encadeamento de geradores
- Assim, podemos facilmente ter um gerador principal e incluir os valores de outros "sub-geradores" em determinados pontos
- Antes do `yield from`, teríamos de fazer isso com `for`:

In [28]:
# SOLUÇÃO COM FOR (SEM YIELD FROM)
def sub_gerador_letras():
    yield "A"
    yield "B"
    yield "C"

def gerador_principal_antigo():
    yield 1
    yield 2
    # Agora, quero incluir os valores do outro gerador:
    for letra in sub_gerador_letras():
        yield letra # Temos que iterar e fazer o 'yield' manualmente
    yield 3
    yield 4

# Testando
g_antigo = gerador_principal_antigo()

for item in g_antigo:
    print(item, end = " ")

1 2 A B C 3 4 

In [33]:
# Agora, com a solução YIELD FROM
def sub_gerador_letras():
    yield "A"
    yield "B"
    yield "C"
    
def gerador_principal_novo():
    yield 1
    yield 2
    yield from sub_gerador_letras()
    yield 3
    yield 4

# Testando
g_novo = gerador_principal_novo()

for item2 in g_novo:
    print(item2, end = " ")

1 2 A B C 3 4 

- Portanto, o `yield from <iteravel>` faz exatamente o que o loop `for` fazia.
- 